In [1]:
# --- 0) Install + imports ---
!pip install -U ultralytics gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
from pathlib import Path
import numpy as np
from PIL import Image
import gradio as gr
import torch
from ultralytics import YOLO


Mounted at /content/drive
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# --- 1) Paths: keep everything under your Drive dataset folder ---
ROOT = Path("/content/drive/My Drive/Colab Notebooks/substation_elements_detection")
RUNS_DIR = ROOT / "runs"                   # where YOLO usually saves results
MODEL_PATH = RUNS_DIR / "yolov8_train" / "weights" / "best.pt"  # your trained weights
FALLBACK = "yolov8n.pt"                    # just in case best.pt isn't there
SAVE_DIR = RUNS_DIR / "gradio_predict"     # we’ll save annotated images here
SAVE_DIR.mkdir(parents=True, exist_ok=True)

In [4]:
# --- 2) Load model once (GPU if available) ---
device = 0 if torch.cuda.is_available() else "cpu"
model = YOLO(MODEL_PATH.as_posix() if MODEL_PATH.exists() else FALLBACK)
print(f"[info] Loaded model from: {MODEL_PATH if MODEL_PATH.exists() else FALLBACK} | device={device}")

[info] Loaded model from: /content/drive/My Drive/Colab Notebooks/substation_elements_detection/runs/yolov8_train/weights/best.pt | device=cpu


In [5]:

# --- 3) Inference helper: returns annotated PIL image ---
def detect(image: np.ndarray, conf: float = 0.25, imgsz: int = 640):
    # image: numpy array (from Gradio)
    # Run prediction (no saving here; we’ll save manually)
    results = model.predict(
        source=image,
        imgsz=int(imgsz),
        conf=float(conf),
        device=device,
        verbose=False
    )
    # Plot first result -> numpy (BGR) -> convert to RGB PIL
    annotated = results[0].plot()                   # np array (BGR)
    annotated_rgb = annotated[:, :, ::-1]          # BGR -> RGB
    pil_img = Image.fromarray(annotated_rgb)


    # Optional: save a copy under runs/gradio_predict
    out_name = SAVE_DIR / "annotated.jpg"
    pil_img.save(out_name)

    return pil_img

In [6]:
# --- 4) Build Gradio UI ---
title = "Substation Elements Detection Tool"
desc = "Upload an image of a substation. The model will annotate detected components (CBDS, FSW, PPI, POW_T, etc.)."

demo = gr.Interface(
    fn=detect,
    inputs=[
        gr.Image(type="numpy", label="Upload image"),
        gr.Slider(0.05, 0.9, value=0.25, step=0.05, label="Confidence threshold"),
        gr.Slider(320, 1280, value=640, step=32, label="Image size (imgsz)")
    ],
    outputs=gr.Image(type="pil", label="Annotated output"),
    title=title,
    description=desc,
)

# Launch (set share=True if you want a public link)
demo.queue().launch(debug=False, share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>